<font color="blue">Привет. Поздравляю с первой работой в ML. Давай смотреть что у тебя получилось

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [116]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('/datasets/users_behavior.csv')
print(df.head())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [118]:
df.shape

(3214, 5)

<font color="blue">Отлично

## 2. Разбейте данные на выборки

разбиваю данные на три выборки  
train - 60%  
test - 20%  
valid - 20%  

<font color="green"> Изменил разбиение  
Вначале разбиваю на  
train - 60%  
other - 40%  
потом разбиваю `other` пополам

In [119]:
#train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
train, other = train_test_split(df, test_size=0.4, random_state=12345)

In [120]:
train

,calls,minutes,messages,mb_used,is_ultra
3027,60.0,431.56,26.0,14751.26,0
434,33.0,265.17,59.0,17398.02,0
1226,52.0,341.83,68.0,15462.38,0
1054,42.0,226.18,21.0,13243.48,0
1842,30.0,198.42,0.0,8189.53,0
...,...,...,...,...,...
2817,12.0,86.62,22.0,36628.85,1
546,65.0,458.46,0.0,15214.25,1
382,144.0,906.18,0.0,25002.44,1
2177,38.0,301.27,37.0,28914.24,1


In [121]:
other

,calls,minutes,messages,mb_used,is_ultra
1415,82.0,507.89,88.0,17543.37,1
916,50.0,375.91,35.0,12388.40,0
1670,83.0,540.49,41.0,9127.74,0
686,79.0,562.99,19.0,25508.19,1
2951,78.0,531.29,20.0,9217.25,0
...,...,...,...,...,...
1650,77.0,481.43,85.0,9765.00,1
1214,83.0,642.19,55.0,10806.69,0
2876,59.0,388.81,27.0,19021.86,1
2010,110.0,796.32,119.0,22339.22,1


In [122]:
test, validate = train_test_split(other, test_size=0.5, random_state=12345)

In [123]:
test

,calls,minutes,messages,mb_used,is_ultra
1386,92.0,536.96,18.0,20193.90,0
3124,40.0,286.57,17.0,17918.75,0
1956,81.0,531.22,56.0,17755.06,0
2286,67.0,460.76,27.0,16626.26,0
3077,22.0,120.09,16.0,9039.57,0
...,...,...,...,...,...
1999,56.0,398.45,4.0,23682.94,0
1023,76.0,601.10,0.0,17104.36,0
748,81.0,525.97,15.0,18878.91,0
1667,10.0,63.03,0.0,2568.00,1


In [124]:
validate

,calls,minutes,messages,mb_used,is_ultra
160,61.0,495.11,8.0,10891.23,0
2498,80.0,555.04,28.0,28083.58,0
1748,87.0,697.23,0.0,8335.70,0
1816,41.0,275.80,9.0,10032.39,0
1077,60.0,428.49,20.0,29389.52,1
...,...,...,...,...,...
2401,55.0,446.06,79.0,26526.28,0
2928,102.0,742.65,58.0,16089.24,1
1985,52.0,349.94,42.0,12150.72,0
357,39.0,221.18,59.0,17865.23,0


In [125]:
train_features = train.drop(['is_ultra'], axis=1)
train_target = train['is_ultra']

In [126]:
valid_features = validate.drop(['is_ultra'], axis=1)
valid_target = validate['is_ultra']

In [127]:
test_features = test.drop(['is_ultra'], axis=1)
test_target = test['is_ultra']

<font color="blue">Странно, импортируешь ``train_test_split`` но бьёшь через ``np.split``. Лучше использовать первый вариант, ну или сделать кросс проверки на отсутсвие пересечений. Давай попрвим эту часть

<font color="blue">Да, теперь ок.

## 3. Исследуйте модели

обучаю модели на train и проверяю valid

Рассмотрим дерево решений   
изменяю глубину в дереве решений

In [128]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)

    model.fit(train_features, train_target)
    predictions_valid = model.predict(valid_features)

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(valid_target, predictions_valid))

max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255


Рассмотрим алгоритм классификации — случайный лес  
изменяю количество деревьев в лесу

In [129]:
for estim in range(10, 71, 10):
    model_RFC = RandomForestClassifier(random_state=12345, n_estimators=estim)
    
    model_RFC.fit(train_features, train_target)   
    predict_RFC = model_RFC.predict(valid_features)
    
    print("n_estimators =", estim, ": ", end='')
    print(accuracy_score(valid_target,predict_RFC))

n_estimators = 10 : 0.7807153965785381
n_estimators = 20 : 0.7822706065318819
n_estimators = 30 : 0.7853810264385692
n_estimators = 40 : 0.7931570762052877
n_estimators = 50 : 0.7931570762052877
n_estimators = 60 : 0.7947122861586314
n_estimators = 70 : 0.7931570762052877


Рассмотрим алгоритм классификации — логическая регрессия

In [130]:
model_LR = LogisticRegression(random_state=12345)

model_LR.fit(train_features, train_target)   
predict_LR = model_LR.predict(valid_features)
print(accuracy_score(valid_target, predict_LR))

0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


<font color="blue">Конечно понятно какую ты модель тренируешь, но лучше в комменатриях прям прописать. Аля "Рассмотрим дерево решений" и дальше уже твой текст

## 4. Проверьте модель на тестовой выборке

выбираю случайный лес с `n_estimators=40`  
проверяю модель на test

In [131]:
model_RFC_60= RandomForestClassifier(random_state=12345, n_estimators=60)

In [132]:
model_RFC_60.fit(train_features, train_target)   

predict_RFC_60 = model_RFC_60.predict(test_features)
print(accuracy_score(test_target, predict_RFC_60))

0.7853810264385692


score > 0.75

## 5. (бонус) Проверьте модели на адекватность

решил проверить критерием полноты

<font color="green">после изменения алгоритма разбиения,  `recall_score` изменился с 0,55 на 0,5079

In [133]:
recall_score(test_target, predict_RFC_60)

0.5079365079365079

т.е. как я понимаю что доля объектов положительного класса из всех объектов положительного класса нашел алгоритм.

<font color="blue">Ну вообще можно говорить об отсечки в 0.5. Поэтому 0.55 уже хорошо, но можно лучше. Вообще когда мы с тобой общались в слаке я думал немного о другом. Итак, мы строим тупую модель. Она что-то будет возращать. Т.к. модель тупая пусть она возращает нечто очень тривиальное. На эту модель тоже можно посчитать accuracу. Ну и потом сравнить. Фокус задачи не в расчёте и в сравнении параметров, это не так сложно. Фокус задачи в объяснении почему так и объяснении порога сравнения.

<font color="green"> Как я понял, мы должны оценить возможжны ли такие ответы в принципе и дать как бы оценку сверху или снизу или порядок значений ответов. Но как это сделать я не догоняю))) 

<font color="blue">Ок, разъясню. В выборке классы не сбалансированы. Давай представим что ты выдаешь константу доминантного класса. Алгоритм у тебя сразу покажет accuracy с его процентной долей. Формально замечаний нет по алгоритму, он основан на тех данных которые у тебя есть. Теперь ты строишь более сложный алгоритм, и он должен быть точнее. Монетка(50%) очень хороший пример т.к. тут 2 класса сбалансированы.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


<font color="blue">В целом хорошая работа. Но надо переделать разбиение данных на использование ``train_test_split``